In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
#import requests

In [2]:
#origin = "WAS"
#destination = "BOS"
startdate = "2020-05-22"
#enddate = "2019-09-09"

url = 'https://www.kayak.com/flights/WAS-BOS/2020-05-22?sort=price_a&fs=equipment=A,R,T,N;stops=~0'

In [9]:
requests = 1
chrome_options = webdriver.ChromeOptions()
agents = ["Firefox/66.0.3","Chrome/73.0.3683.68","Edge/16.16299"]
print("User agent: " + agents[(requests%len(agents))])
chrome_options.add_argument('--user-agent=' + agents[(requests%len(agents))] + '"')    
chrome_options.add_experimental_option('useAutomationExtension', False)
    
driver = webdriver.Chrome("/Users/amac/github/flights/chromedriver", options=chrome_options, desired_capabilities=chrome_options.to_capabilities())
    

#chrome_options = webdriver.ChromeOptions()
#driver = webdriver.Chrome("/Users/amac/github/flights/chromedriver")
driver.implicitly_wait(20)
driver.get(url)
driver.implicitly_wait(20)
link = driver.find_element_by_link_text('Show more results')
ActionChains(browser).move_to_element(link).perform()
link.click()

User agent: Chrome/73.0.3683.68


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Show more results"}
  (Session info: chrome=79.0.3945.130)


In [4]:
soup=BeautifulSoup(driver.page_source, 'lxml')
#print(soup.prettify())

In [5]:
soup=BeautifulSoup(driver.page_source, 'lxml')

deptimes = soup.find_all('span', attrs={'class': 'depart-time base-time'})
arrtimes = soup.find_all('span', attrs={'class': 'arrival-time base-time'})
meridies = soup.find_all('span', attrs={'class': 'time-meridiem meridiem'})
origins = soup.find_all('div', attrs={'class': 'bottom-airport'})
destinations = soup.find_all('div', attrs={'class': 'bottom-airport'})

#print(deptimes)
#print(origins)

deptime = []
for div in deptimes:
    deptime.append(div.getText()[:-1])    
        
arrtime = []
for div in arrtimes:
    arrtime.append(div.getText()[:-1])   
    
origin = []
for div in origins[::2]:
    orig = (div.getText()[:-1]).strip()
    origin.append(orig)
    #origin.append(div.getText()[:-1]) 
    
destination = []
for div in destinations[1::2]:
    dest = (div.getText()[:-1]).strip()
    destination.append(dest)
    #destination.append((div.getText()[:-1]).rstrip("\n\r"))
#print(destination)
 
meridiem = []
for div in meridies:
    meridiem.append(div.getText())  
        
deptime = np.asarray(deptime)
deptime = deptime.reshape(int(len(deptime)), 1)
print(deptime)
    
arrtime = np.asarray(arrtime)
arrtime = arrtime.reshape(int(len(arrtime)), 1)
print(arrtime)
    
meridiem = np.asarray(meridiem)
meridiem = meridiem.reshape(int(len(meridiem)/2), 2)
print(meridiem)

origin = np.asarray(origin)
#origin = origin.reshape(int(len(origin)), 1)
#print(origin.flatten())
print(origin)

#destination = np.asarray(destination)
#destination = destination.reshape(int(len(destination)), 1)
print(destination)

[['7:25']
 ['6:30']
 ['10:00']
 ['8:30']
 ['11:30']
 ['9:30']
 ['12:30']
 ['7:30']
 ['10:30']
 ['7:25']
 ['9:00']
 ['1:30']
 ['2:30']
 ['7:30']
 ['5:00']]
[['3:18']
 ['7:58']
 ['11:29']
 ['10:01']
 ['1:01']
 ['11:03']
 ['2:04']
 ['9:06']
 ['12:06']
 ['8:56']
 ['10:36']
 ['3:14']
 ['4:14']
 ['8:58']
 ['11:41']]
[['am' 'pm']
 ['am' 'am']
 ['pm' 'pm']
 ['am' 'am']
 ['am' 'pm']
 ['am' 'am']
 ['pm' 'pm']
 ['am' 'am']
 ['am' 'pm']
 ['am' 'am']
 ['pm' 'pm']
 ['pm' 'pm']
 ['pm' 'pm']
 ['pm' 'pm']
 ['am' 'am']]
['ZWU' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA' 'DCA'
 'DCA' 'BWI' 'ZWU']
['ZTO', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'BOS', 'ZTO']


In [6]:
regex = re.compile('Common-Booking-MultiBookProvider (.*)multi-row Theme-featured-large(.*)')
price_list = soup.find_all('div', attrs={'class': regex})
    
price = []
for div in price_list:
    #print(div)
    price.append(int(div.getText().split('\n')[3][1:]))
    
print(price)

[81, 109, 144, 144, 144, 144, 144, 144, 144, 174, 174, 174, 174, 192, 192]


In [7]:
df = pd.DataFrame({"Origin" : origin,
                   "Destination" : destination,
                   "Date" : startdate,
                   #"enddate" : enddate,
                   "Price ($)": price,
                   "Currency": "USD",
                   "Depart Time": [m+str(n) for m,n in zip(deptime[:,0],meridiem[:,0])],
                   "Arrive Time": [m+str(n) for m,n in zip(arrtime[:,0],meridiem[:,1])],
                   #"deptime_d": [m+str(n) for m,n in zip(deptime[:,1],meridiem[:,2])],
                   #"arrtime_o": [m+str(n) for m,n in zip(arrtime[:,1],meridiem[:,3])]
                   })

print(df)

   Origin Destination        Date  Price ($) Currency Depart Time Arrive Time
0     ZWU         ZTO  2020-05-22         81      USD      7:25am      3:18pm
1     DCA         BOS  2020-05-22        109      USD      6:30am      7:58am
2     DCA         BOS  2020-05-22        144      USD     10:00pm     11:29pm
3     DCA         BOS  2020-05-22        144      USD      8:30am     10:01am
4     DCA         BOS  2020-05-22        144      USD     11:30am      1:01pm
5     DCA         BOS  2020-05-22        144      USD      9:30am     11:03am
6     DCA         BOS  2020-05-22        144      USD     12:30pm      2:04pm
7     DCA         BOS  2020-05-22        144      USD      7:30am      9:06am
8     DCA         BOS  2020-05-22        144      USD     10:30am     12:06pm
9     DCA         BOS  2020-05-22        174      USD      7:25am      8:56am
10    DCA         BOS  2020-05-22        174      USD      9:00pm     10:36pm
11    DCA         BOS  2020-05-22        174      USD      1:30p